In [27]:
import nflgame
import mirofeatures as mf
import morefeatures as mf2
import pandas as pd
import numpy as np
from sklearn import linear_model

In [29]:
ftrs2, isPass = mf.getFeatures(2009,2016)

In [30]:
df_ftrs = pd.DataFrame(ftrs2)
df_isPass = pd.DataFrame(isPass)
isPass =np.array(isPass)
df_ftrs.head(10)

,down,isHome,opponent,position,quarter,shotgun,team,time,togo
0,1,True,TEN,58,1,0,PIT,893.0,10
1,2,True,TEN,53,1,0,PIT,856.0,5
2,3,True,TEN,56,1,1,PIT,815.0,8
3,1,False,PIT,98,1,0,TEN,796.0,10
4,2,False,PIT,98,1,0,TEN,760.0,10
5,3,False,PIT,94,1,1,TEN,731.0,6
6,1,True,TEN,43,1,0,PIT,684.0,10
7,2,True,TEN,40,1,0,PIT,648.0,7
8,1,True,TEN,30,1,0,PIT,621.0,10
9,2,True,TEN,31,1,0,PIT,583.0,11


In [31]:
ftrs = np.array(ftrs2)

## Using a simple logistic classifier

To start, we have to drop the categorical features because they don't play well with linear classification. 

In [32]:
X = df_ftrs.drop(['opponent', 'team'], axis = 1)

In [58]:
np.random.seed(123456)
mask = np.full(len(ftrs), False)
mask[:len(ftrs)*3/4] = True
np.random.shuffle(mask)

In [50]:
clf = linear_model.SGDClassifier(loss='log')
clf.fit(X[mask],isPass[mask])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [51]:
print("Success rate in predicting pass (for training data) is %f%%" % (100*clf.score(X[mask], isPass[mask])))
print("Success rate in predicting pass (for test data) is %f%%" % (100*clf.score(X[np.invert(mask)], isPass[np.invert(mask)])))

Success rate in predicting pass (for training data) is 59.656662%
Success rate in predicting pass (for test data) is 60.209556%


Before modification: 57% ! Just based on linear regression (not log). Huge amoung of variance based on what our cut is.


In [40]:
np.sum(isPass)/float(len(isPass))

0.57018679877115996

Big surprise! Our naive classifier is only barely better than just always guessing "pass."

Let's compare to a decision tree that Tynan thought up (I know nothing about football so couldn't really help - Miro)

In [41]:
naive = [True for i in range(len(isPass))]
for i in range(len(isPass)):
    if df_ftrs['down'][i] == 1 and df_ftrs['togo'][i] <= 5:
        naive[i] = False
    elif df_ftrs['down'][i] == 2 and df_ftrs['togo'][i] < 6:
        naive[i] = False
    elif df_ftrs['down'][i] == 3 and df_ftrs['togo'][i] < 4:
        naive[i] = False
    elif df_ftrs['down'][i] == 4 and df_ftrs['togo'][i] < 3:
        naive[i] = False

In [42]:
print(np.sum(naive==isPass)/float(len(isPass)))

0.589436669894


Linear regression just barely outperforms this!

## Using LDA (Linear Discriminant Analysis)

In [52]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [59]:
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X[mask],isPass[mask])

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [60]:
print("Success rate in predicting pass (for training data) is %f%%" % (100*lda_model.score(X[mask], isPass[mask])))
print("Success rate in predicting pass (for test data) is %f%%" % (100*lda_model.score(X[np.invert(mask)], isPass[np.invert(mask)])))

Success rate in predicting pass (for training data) is 70.707296%
Success rate in predicting pass (for test data) is 70.990780%


## Using a tree based model

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
rf_model = RandomForestClassifier()
rf_model.fit(X[mask],isPass[mask])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [41]:
print("Success rate in predicting pass (for training data) is %f%%" % (100*rf_model.score(X[mask], isPass[mask])))
print("Success rate in predicting pass (for test data) is %f%%" % (100*rf_model.score(X[np.invert(mask)], isPass[np.invert(mask)])))

Success rate in predicting pass (for training data) is 96.195005%
Success rate in predicting pass (for test data) is 66.618609%


Naively, seems like there is a substantial improvement in predicting play/pass. 66% is not terrible! But 96% training accuracy? Clearly some overfitting is happening here. Let's try dropping out time.


In [46]:
X = df_ftrs.drop(['opponent', 'team', 'time'], axis = 1)

In [47]:
rf_model2 = RandomForestClassifier()
rf_model2.fit(X[mask],isPass[mask])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
print("Success rate in predicting pass (for training data) is %f%%" % (100*rf_model2.score(X[mask], isPass[mask])))
print("Success rate in predicting pass (for test data) is %f%%" % (100*rf_model2.score(X[np.invert(mask)], isPass[np.invert(mask)])))

Success rate in predicting pass (for training data) is 76.204393%
Success rate in predicting pass (for test data) is 69.071249%


Nice! We've increased test accuracy and looks like at least some of the overfitting for training data has been resolved. What if we include team and opponent?

## Expanding our feature set (and also changing data set to be 2002-2012 rather than 2009-2016)

In [2]:
nftrs2, nisPass = mf2.getAllFeatures()

/Users/miro/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (10,19) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
ndf_ftrs = pd.DataFrame(nftrs2)
ndf_isPass = pd.DataFrame(nisPass)
nisPass =np.array(nisPass)
ndf_ftrs.head(10)

,down,isHome,opponent,ptdiff,quarter,shotgun,team,togo,togoal
0,1,True,SF,0,1,0,NYG,10,80
1,2,True,SF,0,1,0,NYG,6,76
2,3,True,SF,0,1,0,NYG,6,76
3,3,True,SF,0,1,1,NYG,11,81
4,1,False,NYG,0,1,0,SF,10,23
5,1,False,NYG,0,1,0,SF,7,7
6,1,False,NYG,0,1,0,SF,12,12
7,2,False,NYG,0,1,0,SF,8,8
8,3,False,NYG,0,1,0,SF,8,8
9,4,False,NYG,0,1,0,SF,7,7


In [13]:
nX = ndf_ftrs.drop(['opponent', 'team','togoal'], axis = 1)

In [23]:
np.random.seed(126)
mask = np.full(len(nftrs2), False)
mask[:len(nftrs2)*3/4] = True
np.random.shuffle(mask)
rf_model3 = RandomForestClassifier()
rf_model3.fit(nX[mask],nisPass[mask])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [24]:
print("Success rate in predicting pass (for training data) is %f%%" % (100*rf_model3.score(nX[mask], nisPass[mask])))
print("Success rate in predicting pass (for test data) is %f%%" % (100*rf_model3.score(nX[np.invert(mask)], nisPass[np.invert(mask)])))

Success rate in predicting pass (for training data) is 73.306311%
Success rate in predicting pass (for test data) is 69.731347%
